In [67]:
!pip install -U langgraph langsmith
!pip install python-dotenv langchain[groq]
!pip install -U langchain-tavily

In [68]:
!pip install streamlit

In [69]:
!pip install streamlit pyngrok




# **BACKEND**

In [70]:
%%writefile backend.py
from google.colab import userdata
import os
from dotenv import load_dotenv

GROQ_API_KEY = os.getenv("groqApiKey")
TAVILY_API_KEY = os.getenv("tavili")
GEMINI_API_KEY = os.getenv("gemini_api_key")


# GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
# TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")


from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.tools import StructuredTool
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
memory = InMemorySaver()


def tavily_formatter(query: str):
    tool = TavilySearch(max_results=1)
    raw = tool.invoke(query)

    # Format clean text
    if raw.get("answer"):
        return raw["answer"]
    elif raw.get("results"):
        first_result = raw["results"][0]
        return f"**{first_result['title']}**\n\n{first_result['content']}\n\n[Read more]({first_result['url']})"
    else:
        return "Sorry, I couldn’t find any useful info."

# Wrap it as a StructuredTool so LangGraph can use it
formatted_tavily = StructuredTool.from_function(
    func=tavily_formatter,
    name="search",
    description="Search the web and return clean human-readable answers."
)


tool = TavilySearch(max_results=1)
tools = [tool]
tools = [formatted_tavily]

llm = init_chat_model("qwen/qwen3-32b", model_provider="groq")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str,thread_id:str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}, {"configurable": {"thread_id": thread_id}}):
        for value in event.values():
            # print("Assistant:", value["messages"][-1].content) # This print is for the notebook, not for streamlit
            yield value["messages"][-1].content





Overwriting backend.py


# **FRONTEND**

In [71]:
%%writefile app.py
import streamlit as st
from backend import stream_graph_updates  # Assuming graph is defined in backend

st.title("Mini AI Bot")

# --- Initialize session state ---
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []   # list of chats
if "current_chat_index" not in st.session_state:
    st.session_state.current_chat_index = None
if "messages" not in st.session_state:
    st.session_state.messages = []
if "thread_ids" not in st.session_state:
    st.session_state.thread_ids = []     # one thread_id per chat
if "next_thread_id" not in st.session_state:
    st.session_state.next_thread_id = 1  # counter for unique threads

# --- Sidebar ---
st.sidebar.header("Chats")

# New Chat button
if st.sidebar.button("🆕 New Chat"):
    # Save current chat if it has content
    if st.session_state.messages:
        if st.session_state.current_chat_index is None:
            st.session_state.chat_history.append(st.session_state.messages.copy())
            st.session_state.thread_ids.append(str(st.session_state.next_thread_id))
        else:
            st.session_state.chat_history[st.session_state.current_chat_index] = st.session_state.messages.copy()

    # Reset for new chat
    st.session_state.messages = []
    st.session_state.current_chat_index = None
    st.session_state.next_thread_id += 1

# Show previous chats
for i, chat in enumerate(st.session_state.chat_history):
    preview = chat[0]["content"][:20] + "..." if chat else f"Chat {i+1}"
    if st.sidebar.button(preview, key=f"chat-{i}"):
        # Save current work before switching
        if st.session_state.current_chat_index is None and st.session_state.messages:
            st.session_state.chat_history.append(st.session_state.messages.copy())
            st.session_state.thread_ids.append(str(st.session_state.next_thread_id))
            st.session_state.next_thread_id += 1
        elif st.session_state.current_chat_index is not None:
            st.session_state.chat_history[st.session_state.current_chat_index] = st.session_state.messages.copy()

        # Load selected chat
        st.session_state.messages = chat.copy()
        st.session_state.current_chat_index = i

# --- Main Chat Area ---
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# --- Chat Input ---
if prompt := st.chat_input("Ask Anything"):
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Determine thread_id for this chat
    if st.session_state.current_chat_index is None:
        thread_id = str(st.session_state.next_thread_id)
    else:
        thread_id = st.session_state.thread_ids[st.session_state.current_chat_index]

    # Stream assistant response
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        assistant_text = ""
        for chunk in stream_graph_updates(prompt, thread_id):
            assistant_text += chunk
            message_placeholder.markdown(assistant_text)

    st.session_state.messages.append({"role": "assistant", "content": assistant_text})

    # Save chat
    if st.session_state.current_chat_index is None:
        st.session_state.chat_history.append(st.session_state.messages.copy())
        st.session_state.thread_ids.append(thread_id)
        st.session_state.current_chat_index = len(st.session_state.chat_history) - 1
        st.session_state.next_thread_id += 1
    else:
        st.session_state.chat_history[st.session_state.current_chat_index] = st.session_state.messages.copy()


Overwriting app.py


In [72]:
from pyngrok import ngrok
import subprocess
from google.colab import userdata
import os

# Load ngrok authtoken from Colab secrets
os.environ["NGROK_AUTHTOKEN"] = userdata.get("NGROK_AUTHTOKEN")

# Start streamlit
# port = 8501
# proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", str(port), "--server.headless", "true"])

# # Open tunnel
# public_url = ngrok.connect(port)
# print("Streamlit app running at:", public_url)

In [73]:
!pkill streamlit
!pkill ngrok

In [74]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('groqApiKey')
os.environ["TAVILY_API_KEY"] = userdata.get('tavili')
port = 8501
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", str(port), "--server.headless", "true"])


In [75]:
public_url = ngrok.connect(port)
print("Streamlit app running at:", public_url)




Streamlit app running at: NgrokTunnel: "https://dbf62f851ede.ngrok-free.app" -> "http://localhost:8501"
